In [31]:
import torch.nn as nn
import torch
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

In [32]:
input_size = 28
num_classes = 10
num_epochs = 100
batch_size = 64
train_dataset = datasets.MNIST(root="./data", train=True, transform=transforms.ToTensor(),download=True)
test_dataset = datasets.MNIST(root="./data", train=False, transform=transforms.ToTensor(), download=True)
device = torch.device("mps")

In [33]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [34]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
        nn.Conv2d(
            in_channels=1,
            out_channels=16,
            kernel_size=5,
            stride=1,
            padding=2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
        )
        
        self.conv2 = nn.Sequential(
        nn.Conv2d(16,32,5,1,2),
        nn.ReLU(),
        nn.MaxPool2d(2),
        )
        
        self.out = nn.Linear(32*7*7, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        output = self.out(x)
        return output

In [35]:
def accuracy(predictions, labels):
    pred = torch.max(predictions.data,1)[1]
    rights = pred.eq(labels.data.view_as(pred)).sum()
    return rights, len(labels)

In [36]:
net = CNN()
net = net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

for epoch in range(num_epochs):
    train_rights = []
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        net.train()
        output = net(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        right = accuracy(output, target)
        train_rights.append(right)
        
        if batch_idx%100 == 0:
            net.eval()
            val_rights = []
            
            for (data, target) in test_loader:
                data, target = data.to(device), target.to(device)
                output = net(data)
                right = accuracy(output, target)
                val_rights.append(right)
                
            train_r = (sum([tup[0] for tup in train_rights]), sum([tup[1] for tup in train_rights]))
            val_r = (sum([tup[0] for tup in val_rights]), sum([tup[1] for tup in val_rights]))
            print("current epoch:{} [{}/{} ({:.0f}%)]\tloss:{:.6f}\taccuracy in train:{:.2f}%\taccuracy in test:{:.2f}%".format(epoch, batch_idx*batch_size, 
                                                                        len(train_loader.dataset),
                                                                        100.*batch_idx/len(train_loader),
                                                                        loss.data,
                                                                        100.*train_r[0].cpu().numpy()/train_r[1],
                                                                        100.*val_r[0].cpu().numpy()/val_r[1]))

current epoch:0 [0/60000 (0%)]	loss:2.301053	accuracy in train:6.25%	accuracy in test:11.30%
current epoch:0 [6400/60000 (11%)]	loss:0.361228	accuracy in train:76.25%	accuracy in test:92.02%
current epoch:0 [12800/60000 (21%)]	loss:0.150717	accuracy in train:84.85%	accuracy in test:95.33%
current epoch:0 [19200/60000 (32%)]	loss:0.111984	accuracy in train:88.46%	accuracy in test:96.53%
current epoch:0 [25600/60000 (43%)]	loss:0.176534	accuracy in train:90.39%	accuracy in test:97.33%
current epoch:0 [32000/60000 (53%)]	loss:0.138096	accuracy in train:91.69%	accuracy in test:97.16%
current epoch:0 [38400/60000 (64%)]	loss:0.068610	accuracy in train:92.53%	accuracy in test:97.20%
current epoch:0 [44800/60000 (75%)]	loss:0.187391	accuracy in train:93.20%	accuracy in test:97.80%
current epoch:0 [51200/60000 (85%)]	loss:0.124814	accuracy in train:93.74%	accuracy in test:98.10%
current epoch:0 [57600/60000 (96%)]	loss:0.034694	accuracy in train:94.17%	accuracy in test:98.24%
current epoch:1 [

current epoch:8 [25600/60000 (43%)]	loss:0.001163	accuracy in train:99.59%	accuracy in test:99.16%
current epoch:8 [32000/60000 (53%)]	loss:0.021815	accuracy in train:99.59%	accuracy in test:99.08%
current epoch:8 [38400/60000 (64%)]	loss:0.003497	accuracy in train:99.57%	accuracy in test:98.72%
current epoch:8 [44800/60000 (75%)]	loss:0.024784	accuracy in train:99.57%	accuracy in test:99.14%
current epoch:8 [51200/60000 (85%)]	loss:0.003477	accuracy in train:99.58%	accuracy in test:98.89%
current epoch:8 [57600/60000 (96%)]	loss:0.008528	accuracy in train:99.58%	accuracy in test:99.03%
current epoch:9 [0/60000 (0%)]	loss:0.013492	accuracy in train:100.00%	accuracy in test:98.97%
current epoch:9 [6400/60000 (11%)]	loss:0.007258	accuracy in train:99.78%	accuracy in test:99.06%
current epoch:9 [12800/60000 (21%)]	loss:0.002894	accuracy in train:99.77%	accuracy in test:99.08%
current epoch:9 [19200/60000 (32%)]	loss:0.005634	accuracy in train:99.77%	accuracy in test:99.21%
current epoch:9

current epoch:16 [44800/60000 (75%)]	loss:0.000560	accuracy in train:99.86%	accuracy in test:99.07%
current epoch:16 [51200/60000 (85%)]	loss:0.000292	accuracy in train:99.86%	accuracy in test:99.14%
current epoch:16 [57600/60000 (96%)]	loss:0.029922	accuracy in train:99.86%	accuracy in test:99.11%
current epoch:17 [0/60000 (0%)]	loss:0.071678	accuracy in train:98.44%	accuracy in test:99.13%
current epoch:17 [6400/60000 (11%)]	loss:0.074199	accuracy in train:99.77%	accuracy in test:99.04%
current epoch:17 [12800/60000 (21%)]	loss:0.000662	accuracy in train:99.84%	accuracy in test:99.10%
current epoch:17 [19200/60000 (32%)]	loss:0.000122	accuracy in train:99.85%	accuracy in test:99.15%
current epoch:17 [25600/60000 (43%)]	loss:0.000371	accuracy in train:99.85%	accuracy in test:99.13%
current epoch:17 [32000/60000 (53%)]	loss:0.000963	accuracy in train:99.87%	accuracy in test:99.08%
current epoch:17 [38400/60000 (64%)]	loss:0.000109	accuracy in train:99.85%	accuracy in test:98.96%
curren

current epoch:25 [0/60000 (0%)]	loss:0.000016	accuracy in train:100.00%	accuracy in test:98.77%
current epoch:25 [6400/60000 (11%)]	loss:0.000776	accuracy in train:99.83%	accuracy in test:99.03%
current epoch:25 [12800/60000 (21%)]	loss:0.000007	accuracy in train:99.88%	accuracy in test:99.13%
current epoch:25 [19200/60000 (32%)]	loss:0.000004	accuracy in train:99.87%	accuracy in test:99.08%
current epoch:25 [25600/60000 (43%)]	loss:0.000112	accuracy in train:99.88%	accuracy in test:99.05%
current epoch:25 [32000/60000 (53%)]	loss:0.001186	accuracy in train:99.88%	accuracy in test:99.02%
current epoch:25 [38400/60000 (64%)]	loss:0.000008	accuracy in train:99.88%	accuracy in test:99.11%
current epoch:25 [44800/60000 (75%)]	loss:0.000035	accuracy in train:99.88%	accuracy in test:99.17%
current epoch:25 [51200/60000 (85%)]	loss:0.000227	accuracy in train:99.88%	accuracy in test:99.18%
current epoch:25 [57600/60000 (96%)]	loss:0.000005	accuracy in train:99.88%	accuracy in test:99.10%
curre

current epoch:33 [19200/60000 (32%)]	loss:0.000002	accuracy in train:99.92%	accuracy in test:99.22%
current epoch:33 [25600/60000 (43%)]	loss:0.000027	accuracy in train:99.93%	accuracy in test:98.96%
current epoch:33 [32000/60000 (53%)]	loss:0.000031	accuracy in train:99.92%	accuracy in test:99.02%
current epoch:33 [38400/60000 (64%)]	loss:0.056439	accuracy in train:99.91%	accuracy in test:98.99%
current epoch:33 [44800/60000 (75%)]	loss:0.000040	accuracy in train:99.91%	accuracy in test:99.18%
current epoch:33 [51200/60000 (85%)]	loss:0.000449	accuracy in train:99.89%	accuracy in test:99.12%
current epoch:33 [57600/60000 (96%)]	loss:0.002382	accuracy in train:99.90%	accuracy in test:99.21%
current epoch:34 [0/60000 (0%)]	loss:0.005704	accuracy in train:100.00%	accuracy in test:99.19%
current epoch:34 [6400/60000 (11%)]	loss:0.000016	accuracy in train:99.94%	accuracy in test:99.18%
current epoch:34 [12800/60000 (21%)]	loss:0.000000	accuracy in train:99.95%	accuracy in test:99.27%
curre

...

current epoch:66 [25600/60000 (43%)]	loss:0.000000	accuracy in train:100.00%	accuracy in test:99.20%
current epoch:66 [32000/60000 (53%)]	loss:0.000000	accuracy in train:100.00%	accuracy in test:99.21%
current epoch:66 [38400/60000 (64%)]	loss:0.000000	accuracy in train:100.00%	accuracy in test:99.21%
current epoch:66 [44800/60000 (75%)]	loss:0.000000	accuracy in train:100.00%	accuracy in test:99.18%
current epoch:66 [51200/60000 (85%)]	loss:0.000000	accuracy in train:100.00%	accuracy in test:99.18%
current epoch:66 [57600/60000 (96%)]	loss:0.000000	accuracy in train:100.00%	accuracy in test:99.20%
current epoch:67 [0/60000 (0%)]	loss:0.000000	accuracy in train:100.00%	accuracy in test:99.19%
current epoch:67 [6400/60000 (11%)]	loss:0.000000	accuracy in train:100.00%	accuracy in test:99.20%
current epoch:67 [12800/60000 (21%)]	loss:0.000000	accuracy in train:100.00%	accuracy in test:99.19%
current epoch:67 [19200/60000 (32%)]	loss:0.000000	accuracy in train:100.00%	accuracy in test:99.

current epoch:74 [38400/60000 (64%)]	loss:0.000000	accuracy in train:99.96%	accuracy in test:99.22%
current epoch:74 [44800/60000 (75%)]	loss:0.000000	accuracy in train:99.97%	accuracy in test:99.10%
current epoch:74 [51200/60000 (85%)]	loss:0.000000	accuracy in train:99.97%	accuracy in test:99.23%
current epoch:74 [57600/60000 (96%)]	loss:0.102410	accuracy in train:99.97%	accuracy in test:99.27%
current epoch:75 [0/60000 (0%)]	loss:0.000000	accuracy in train:100.00%	accuracy in test:99.15%
current epoch:75 [6400/60000 (11%)]	loss:0.000002	accuracy in train:99.98%	accuracy in test:99.07%
current epoch:75 [12800/60000 (21%)]	loss:0.000000	accuracy in train:99.97%	accuracy in test:99.20%
current epoch:75 [19200/60000 (32%)]	loss:0.000000	accuracy in train:99.96%	accuracy in test:98.99%
current epoch:75 [25600/60000 (43%)]	loss:0.000033	accuracy in train:99.96%	accuracy in test:99.10%
current epoch:75 [32000/60000 (53%)]	loss:0.000004	accuracy in train:99.97%	accuracy in test:99.07%
curre

current epoch:82 [57600/60000 (96%)]	loss:0.000171	accuracy in train:99.97%	accuracy in test:99.21%
current epoch:83 [0/60000 (0%)]	loss:0.000003	accuracy in train:100.00%	accuracy in test:99.22%
current epoch:83 [6400/60000 (11%)]	loss:0.000000	accuracy in train:100.00%	accuracy in test:99.22%
current epoch:83 [12800/60000 (21%)]	loss:0.000000	accuracy in train:100.00%	accuracy in test:99.23%
current epoch:83 [19200/60000 (32%)]	loss:0.000010	accuracy in train:100.00%	accuracy in test:99.20%
current epoch:83 [25600/60000 (43%)]	loss:0.000000	accuracy in train:100.00%	accuracy in test:99.18%
current epoch:83 [32000/60000 (53%)]	loss:0.000001	accuracy in train:100.00%	accuracy in test:99.17%
current epoch:83 [38400/60000 (64%)]	loss:0.000000	accuracy in train:100.00%	accuracy in test:99.22%
current epoch:83 [44800/60000 (75%)]	loss:0.000003	accuracy in train:100.00%	accuracy in test:99.16%
current epoch:83 [51200/60000 (85%)]	loss:0.000189	accuracy in train:100.00%	accuracy in test:99.1

current epoch:91 [12800/60000 (21%)]	loss:0.000000	accuracy in train:99.97%	accuracy in test:99.08%
current epoch:91 [19200/60000 (32%)]	loss:0.000000	accuracy in train:99.96%	accuracy in test:99.08%
current epoch:91 [25600/60000 (43%)]	loss:0.000000	accuracy in train:99.96%	accuracy in test:99.17%
current epoch:91 [32000/60000 (53%)]	loss:0.000025	accuracy in train:99.97%	accuracy in test:99.15%
current epoch:91 [38400/60000 (64%)]	loss:0.000000	accuracy in train:99.97%	accuracy in test:99.17%
current epoch:91 [44800/60000 (75%)]	loss:0.000004	accuracy in train:99.97%	accuracy in test:98.88%
current epoch:91 [51200/60000 (85%)]	loss:0.000000	accuracy in train:99.96%	accuracy in test:99.14%
current epoch:91 [57600/60000 (96%)]	loss:0.000000	accuracy in train:99.96%	accuracy in test:99.04%
current epoch:92 [0/60000 (0%)]	loss:0.000005	accuracy in train:100.00%	accuracy in test:99.06%
current epoch:92 [6400/60000 (11%)]	loss:0.002311	accuracy in train:99.92%	accuracy in test:98.98%
curre

current epoch:99 [32000/60000 (53%)]	loss:0.000001	accuracy in train:99.90%	accuracy in test:99.19%
current epoch:99 [38400/60000 (64%)]	loss:0.000052	accuracy in train:99.91%	accuracy in test:99.13%
current epoch:99 [44800/60000 (75%)]	loss:0.000038	accuracy in train:99.92%	accuracy in test:99.13%
current epoch:99 [51200/60000 (85%)]	loss:0.000000	accuracy in train:99.92%	accuracy in test:99.15%
current epoch:99 [57600/60000 (96%)]	loss:0.000001	accuracy in train:99.92%	accuracy in test:99.21%


In [ ]:
for data, label in train_dataset:
    plt.imshow(data.numpy().reshape(28,28),cmap="Greys")

In [ ]:
plt.show()